## Capstone Project Title : AI-Driven Vendor Governance and Risk Intelligence System for Telecom Operations

### Problem Statement : 

Telecom projects face recurring vendor delays due to operational, commercial, and compliance-related factors. Existing governance models are reactive and escalation-driven, resulting in late interventions and inefficient decision-making. This project aims to build an AI-enabled vendor governance and decision support system that predicts delivery risks early, identifies root causes, and recommends targeted actions to Operations and SCM teams, thereby enabling data-driven vendor governance.

### Project Scope :

- Synthetic telecom vendor dataset
- Vendor Performance Index (VPI)
- ML-based delay prediction
- Root cause identification
- Severity classification
- AI-assisted action recommendations
- Management clarity framework

### Success Criteria :

- Predict delay risk with reasonable accuracy
- Explain why a vendor is risky
- Recommend what action should be taken
- Replace escalation logic with decision logic

## Phase 1 : Dataset Design & Feature Engineering
Data Sources (Synthetic)
### STEP 1 — Synthetic Dataset Creation
    A single, realistic CSV dataset that simulates telecom vendor governance reality.

### Dataset Schema :

1. Vendor Master
   | Column                  | Description                 |
| ----------------------- | --------------------------- |
| Vendor_ID               | Unique vendor identifier    |
| Vendor_Type             | Fiber / Civil / O&M         |
| Region                  | North / South / East / West |
| Vendor_Experience_Years | Years working with operator |

2. Operational Metrics
| Column                | Description                      |
| --------------------- | -------------------------------- |
| Planned_Delivery_Days | Contractual timeline             |
| Actual_Delivery_Days  | Actual execution days            |
| Delay_Days            | Actual − Planned                 |
| Site_Readiness_Issues | Count of site readiness problems |
| Rework_Percentage     | % work redone                    |

3. SCM / Commercial Metrics
| Column                   | Description       |
| ------------------------ | ----------------- |
| Payment_Cycle_Days       | Avg payment cycle |
| Invoice_Disputes         | Count of disputes |
| Contract_Penalty_Invoked | 0 / 1             |

4. Quality & Compliance
| Column                  | Description               |
| ----------------------- | ------------------------- |
| Quality_Score           | 0–100                     |
| Compliance_Issues_Count | Statutory / safety issues |


### This dataset will support:

VPI scoring
Delay prediction (ML)
Root cause analysis
Action recommendation

                

## Phase 2: Data Cleaning & Manipulation

In [21]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import joblib
import sys

In [22]:
# ==========================================================
# 0) Config
# ==========================================================
base_path = r"C:\Users\hp\Project_Vendor_Governance"

import os
os.makedirs(f"{base_path}\\processed", exist_ok=True)
os.makedirs(f"{base_path}\\models", exist_ok=True)

# ==========================================================
# 1) Load raw datasets
# ==========================================================
vendor_master = pd.read_csv(f"{base_path}\\data\\raw\\vendor_master_scaled.csv")
delivery = pd.read_csv(f"{base_path}\\data\\raw\\vendor_delivery_history_scaled.csv")
quality = pd.read_csv(f"{base_path}\\data\\raw\\vendor_quality_compliance_scaled.csv")
payments = pd.read_csv(f"{base_path}\\data\\raw\\vendor_payments_scm_scaled.csv")

# Optional: force numeric types (prevents hidden string issues)
for c in ["delivery_delay_days", "planned_days"]:
    if c in delivery.columns:
        delivery[c] = pd.to_numeric(delivery[c], errors="coerce")

for c in ["payment_delay_days", "invoice_amount_lakhs"]:
    if c in payments.columns:
        payments[c] = pd.to_numeric(payments[c], errors="coerce")

for c in ["rejection_rate_pct", "penalty_cases"]:
    if c in quality.columns:
        quality[c] = pd.to_numeric(quality[c], errors="coerce")

# ------------------ Data Cleaning ------------------
# Ensure no negative delays
delivery["delivery_delay_days"] = delivery["delivery_delay_days"].clip(lower=0)  #replaces any value below zero with zero
payments["payment_delay_days"] = payments["payment_delay_days"].clip(lower=0)  #replaces any value below zero with zero

# ------------------ Delivery Aggregation ------------------
delivery["delivery_month"] = pd.to_datetime(delivery["delivery_month"], dayfirst=True)  #converts to date-time format
delivery_sorted = delivery.sort_values(["vendor_id", "delivery_month"])   #Sorts the data first by vendor_id then by delivery_month

delivery_agg = delivery_sorted.groupby("vendor_id").agg(                     #Groups the entire dataset vendor-wise
    avg_delay_days=("delivery_delay_days", "mean"),                                   #Compute Average delay
    max_delay_days=("delivery_delay_days", "max"),                                    #Compute Max delay
    delay_frequency=("delivery_delay_days", lambda x: (x > 0).mean()),                #Calculates the frequency of delay
    avg_planned_days=("planned_days", "mean")                                #promised by vendor
).reset_index()                   #After groupby, vendor_id becomes an index .reset_index() converts it back into a normal column

# ------------------ Quality Transformation ------------------

quality_agg = quality.groupby("vendor_id").agg(
    rejection_rate_pct=("rejection_rate_pct", "mean"),
    penalty_cases=("penalty_cases", "sum"),
).reset_index()

# ------------------ Payments Aggregation ------------------
payments["payment_risk_flag"] = (payments["payment_delay_days"] > 15).astype(int)

payments_agg = payments.groupby("vendor_id").agg(
    avg_payment_delay=("payment_delay_days", "mean"),
    payment_risk_ratio=("payment_risk_flag", "mean"),
    total_invoice_amount=("invoice_amount_lakhs", "sum")
).reset_index()

# ------------------ Merge All ------------------
model_ready_df = (
    vendor_master
    .merge(delivery_agg, on="vendor_id", how="left")
    .merge(quality_agg, on="vendor_id", how="left")
    .merge(payments_agg, on="vendor_id", how="left")
)

# Handle remaining missing values
model_ready_df.fillna(0, inplace=True)

# Save final dataset
final_path = f"{base_path}\\processed\\vendor_model_ready_dataset.csv"
model_ready_df.to_csv(final_path, index=False)

print("Saved:",final_path)


Saved: C:\Users\hp\Project_Vendor_Governance\processed\vendor_model_ready_dataset.csv


## PHASE 3 — VENDOR PERFORMANCE INDEX (VPI)

Objective :

Convert multiple vendor performance dimensions into one interpretable score (0–100) that management can trust.

In [23]:
##Load Model ready dataset

from sklearn.preprocessing import MinMaxScaler

# Load processed dataset
df = pd.read_csv(f"{base_path}\\processed\\vendor_model_ready_dataset.csv")

df.head()

,vendor_id,vendor_name,vendor_type,msme_flag,location,contract_value_lakhs,relationship_years,avg_delay_days,max_delay_days,delay_frequency,avg_planned_days,rejection_rate_pct,penalty_cases,avg_payment_delay,payment_risk_ratio,total_invoice_amount
0,V0001,Vendor_V0001,Infra,0,Hyderabad,375,5,2.375000,9,0.625000,36.958333,5.59,0,5.400000,0.000000,843
1,V0002,Vendor_V0002,Services,0,Pune,343,12,2.666667,7,0.750000,35.583333,1.84,1,6.782609,0.130435,1345
2,V0003,Vendor_V0003,OEM,0,Chennai,152,11,1.500000,8,0.458333,33.916667,7.15,2,5.954545,0.045455,1438
3,V0004,Vendor_V0004,OEM,1,Pune,146,11,3.708333,12,0.833333,35.750000,6.45,0,7.071429,0.071429,1771
4,V0005,Vendor_V0005,Infra,1,Hyderabad,125,1,4.291667,13,0.750000,33.000000,2.34,5,6.285714,0.000000,1152


In [24]:
# ==========================================================
# Compute VPI Score + Category
# ==========================================================
required_cols = [
    "avg_delay_days", "delay_frequency", "avg_payment_delay",
    "payment_risk_ratio", "rejection_rate_pct", "penalty_cases"
]
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"Missing required columns in model_ready dataset: {missing}")

# -----------------------------
# 1. Select clean VPI features
# -----------------------------

vpi_features = df[
    [
        "avg_delay_days",        # Delivery
        "delay_frequency",       # Delivery consistency
        "avg_payment_delay",     # Payment
        "payment_risk_ratio",    # Payment discipline
        "rejection_rate_pct",    # Quality
        "penalty_cases"          # Compliance
    ]
].copy()

# -----------------------------
# 2. Normalize (0–1 scale)
# -----------------------------

scaler = MinMaxScaler()
vpi_scaled = pd.DataFrame(
    scaler.fit_transform(vpi_features),
    columns=vpi_features.columns,
    index=df.index
)

# -----------------------------
# 3. Invert risk metrics
# (Lower value = better vendor)
# -----------------------------

invert_cols = vpi_scaled.columns.tolist()

for col in invert_cols:
    vpi_scaled[col] = 1 - vpi_scaled[col]

# -----------------------------
# 4. Compute VPI Score
# -----------------------------

df["VPI_Score"] = (
    vpi_scaled["avg_delay_days"] * 0.20 +
    vpi_scaled["delay_frequency"] * 0.15 +
    vpi_scaled["avg_payment_delay"] * 0.15 +
    vpi_scaled["payment_risk_ratio"] * 0.10 +
    vpi_scaled["rejection_rate_pct"] * 0.25 +
    vpi_scaled["penalty_cases"] * 0.15
) * 100

# -----------------------------
# 5. Categorize Vendors
# -----------------------------

def vpi_category(score):
    if score >= 75:
        return "High"
    elif score >= 50:
        return "Medium"
    else:
        return "Low"

df["VPI_Category"] = df["VPI_Score"].apply(vpi_category)

# -----------------------------
# 6. Save Output
# -----------------------------

output_path = f"{base_path}\\processed\\vendor_vpi_scores.csv"
df.to_csv(output_path, index=False)

print("VPI file saved at:", output_path)


VPI file saved at: C:\Users\hp\Project_Vendor_Governance\processed\vendor_vpi_scores.csv


### Phase 3: Delay Prediction Model
Predict delay_days for a vendor transaction before payment happens.

Target Variable - delay_days

Features (X): 
delivery_delay_days
quality_score
compliance_score
payment_amount
vendor_risk_score
past_delay_avg
delay_frequency

This is Regression problem - Numeric output - continous

## Phase 3A — Create Transaction-Level Dataset (Delay Modeling)

Create one row per invoice/payment with a real target variable (delay_days) so that:

ML can learn patterns
Temporal behavior is preserved
Aggregation bias is avoided


In [26]:
sys.path.append(r"C:\Users\hp\Project_Vendor_Governance")

In [27]:
from src.risk_model import train_and_save_model
train_and_save_model(
    r"C:\Users\hp\Project_Vendor_Governance\processed\vendor_vpi_scores.csv",
    model_path=r"C:\Users\hp\Project_Vendor_Governance\models\risk_model.pkl"  
)


Model Performance
              precision    recall  f1-score   support

           0       0.87      0.96      0.91        55
           1       0.93      0.77      0.84        35

    accuracy                           0.89        90
   macro avg       0.90      0.87      0.88        90
weighted avg       0.89      0.89      0.89        90

ROC-AUC: 0.9703896103896104
Model saved at: C:\Users\hp\Project_Vendor_Governance\models\risk_model.pkl


,n_estimators,200
,criterion,'gini'
,max_depth,6
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [28]:
import joblib

bundle = joblib.load(r"C:\Users\hp\Project_Vendor_Governance\models\risk_model.pkl")
bundle["features"]

artifact = joblib.load(r"C:\Users\hp\Project_Vendor_Governance\models\risk_model.pkl")
print(type(artifact))

<class 'dict'>


## Combine Delay Probability + VPI → Vendor Risk Score

Objective

Combine:
Delay probability (ML output)
Vendor health (VPI score)

Into one interpretable Vendor Risk Score and severity bucket.

## PHASE 4 — Root Cause Identification (Explainable AI)

Objective
- For each risky vendor, identify:
- Top drivers contributing to delay risk
- In business language, not ML jargon
- This answers management’s question:

“WHY is this vendor risky?”

# PHASE 6 — Action Recommendation Engine

### Move governance from:

❌ “Escalate vendor”
to
✅ “Do THIS action to reduce risk”

## Define Action Mapping (Business Logic)

### 🛠 Operations Actions

## PHASE 7 — Management KPI & Decision View

🎯 Objective

Convert detailed AI outputs into simple, board-ready insights:
- Where are the risks?
- Why are they happening?
- What actions are underway?
- What impact can be expected?